# Python functions

**Coding party OpenTURNS, march 2023**

Michaël Baudin

Mathieu Couplet


## Abstract

In this Notebook, we present scripts to connect OpenTURNS to a Python function. This is required to propagate the uncertainties through the function. 

## References

* User Manual, Functions [Lien](http://openturns.github.io/openturns/master/user_manual/functions.html)
* Examples, Functional modeling : [Lien](http://openturns.github.io/openturns/master/examples/functional_modeling/functional_modeling.html)

* Classe MemoizeFunction : [Lien](http://openturns.github.io/openturns/master/user_manual/_generated/openturns.MemoizeFunction.html)

* Sur ExprTk : OpenTURNS Users’ Day #11, Friday, the 15 th, June 2018, Denis Barbier, [Lien](http://trac.openturns.org/blog/OpenTURNS_Users_Day_11)

## Use a function

The most common way to use a function is as follows:

* we create a `Function`,
* we use it to feed a `RandomVector`.

Example :

```python
[...]
myWrapper = PythonFunction ( MyWrapperClass ())
[...]
outVariable = CompositeRandomVector ( myWrapper , inRandomVector )
[...]
```

Other uses of Python functions:

* Bayesian calibration: see `RandomWalkMetropolisHastings`
* stochastic processes : see `FieldFunction`
* optimization : see `OptimizationProblem`
* parametric functions : see `ParametricFunction`

## A mathematical function

For each case, we use the following mathematical problem:

* 3 inputs, with standard normal and independent, 
* 2 outputs.

We will consider the symbolic formula :
$$
\begin{eqnarray}
Y_1 &=& X_1 + X_2 + X_3 \\
Y_2 &=& X_1 − X_2 X_3
\end{eqnarray}
$$

The exact results are presented in the following table.

| Variable | Expectation | Standard deviation |
|-|-|-|
| $Y_1$ | 0 | 1.732 |
| $Y_2$ | 0 | 1.415 |

**Table 1.** Expectation and standard deviation of the output of the mathematical model.


In [1]:
import openturns as ot
import numpy as np
import time
import math
import multiprocessing

ot.__version__

'1.19'

In [2]:
X0 = ot.Normal(0.0, 1.0)
X1 = ot.Normal(0.0, 1.0)
X2 = ot.Normal(0.0, 1.0)
inputDistribution = ot.ComposedDistribution((X0, X1, X2))
inputRandomVector = ot.RandomVector(inputDistribution)

## PythonFunction : constructor

The `PythonFunction` classe creates function that OpenTURNS can evaluate. It uses a Python function defined by the `def` keyword.

![PythonFunction](PythonFunction.png)

The constructor of the class is:

`PythonFunction ( nbInputs , nbOutputs , myPythonFunc )`

where:

* `nbInputs`: number of input variables, 
* `nbOutputs`: number of output variables,
* `myPythonFunc`: a Python function.

The function `mySimulator` has the calling sequence `y=mySimulator(x)` where:

* `x`: the input of the simulator, a vector with dimension `nbInputs`,
* `y`: la sortie du simulateur, a vector with dimension `nbOutputs`.

In [3]:
def mySimulator(x):
    y0 = x[0] + x[1] + x[2]
    y1 = x[0] - x[1] * x[2]
    y = [y0, y1]
    return y

## Why do I have to use the PythonFunction class... in Python???

For a Python user of the `scipy`, `scikit-learn` or any other Python library, it may seem unnecessary to convert a Python function into a `PythonFunction`. There are several reasons why this is necessary and can be useful.

OpenTURNS is a C++ library that we can access using Python. Hence, the library cannot evaluate a Python function directly. The SWIG layer of OpenTURNS allows Python users to access to OpenTURNS using Python. Hence, the main purpose of the `PythonFunction` class is to convert the Python function into a `Function` than OpenTURNS can evaluate.

At this point, some Python users may require that the conversion is performed automatically, so that the user would not have to do it explicitly. This may be an option in a library that only uses Python functions. But OpenTURNS provides several types of functions (8 different types of Python functions) and each type of function has its own purpose: for example, the `PythonFieldFunction` manages a spatial or temporal field function. Providing an automatic conversion would create confusing scripts where the specific type of object would be unknown when reading the script. 

Finally, most libraries do not require to provide the `PythonFunction` class, because the Python function is the only way to provide a function. This may lead to inefficient algorithms. Indeed, the Python function defined by `def` has poor semantic content:

- is the function linear, quadratic?
- is the function parametric?
- is the function composed?
- is the function part of an orthonormal family of functions?
- has the function an implementation of the gradient, of the Hessian?

In OpenTURNS, there are many ways to create a function and to combine them with other objects. For example, the `SymbolicFunction` can create a function using a string which defines the function. This function can be made parametric using the `ParametricFunction`, composed to another function using the `ComposedFunction` class, etc. The OpenTURNS libraries provides more than 50 different ways to create and combine functions, so that each algorithm can use the specific feature of each type of function. For example, the `DistanceToDomainFunction` can be used to estimate HSIC indices. The `LinearFunction` is a linear function that can be used e.g. in an adaptive directional stratification algorithm when the probability to compute is defined as a combination of hyperplanes. In other words, the `PythonFunction` class in OpenTURNS illustrates the fact that many different types of functions can be defined, which makes so that the very specific properties of each function can be defined by the user. This enables to get efficient and potentially fast algorithms, because the specific property of each function is known by the algorithm through the class it derives from.

## Examples of the PythonFunction class

In the next example, we estimate the mean of the output using a Monte-Carlo sample with size equal to 10000 observations.

In [4]:
myWrapper = ot.PythonFunction(3, 2, mySimulator)
outputVect = ot.CompositeRandomVector(myWrapper, inputRandomVector)
montecarlosize = 10000
outputSample = outputVect.getSample(montecarlosize)
empiricalMean = outputSample.computeMean()
print(empiricalMean)
empiricalSd = outputSample.computeStandardDeviation()
print(empiricalSd)

[-0.0166778,-0.0123527]
[1.73359,1.39888]


## What type for x and y?

| Type | Input X | Output Y |
|-|-|-|
| list (Python) | | ✓ |
| tuple (Python) | | ✓ |
| array (NumPy) | | ✓ |
| Point (OpenTURNS) | ✓ | ✓ |

**Table 2.** The different types of inputs and outputs of a `PythonFunction`.

## PythonFunction : goals, advantages, drawbacks

The goals of the `PythonFunction` are:

* to be simple to implement, 
* to use the flexibility of Python: enables to use any Python package to evaluate the output (e.g. `scipy`).

Advantages:

* Useful if the simulation already is in Python or has a Python API.
* Can be combined with "Coupling Tools" to connect to an external program through files.
* Can be vectorised with `func_sample`.
* Can be parallelized on multiple cpus with the `n_cpus` option (see l'exercise 5).

Drawbacks:

* The derivatives (gradient, Hessian) uses, by default, finite differences. This can lead to inaccurate results if the step size is poorly chosen.

## Vectorised PythonFunction : goals, advantages and drawbacks

The `PythonFunction` class has a `func_sample` option:

* Idea: improve the performance by vectorising operations.
* Principle: evaluate all the outputs in the `Sample` with a single evaluation, without a `for` loop.
* Implementation: the input and output are `Sample` instead of a `Point`.

Advantage:

* Improve performance

Drawback:

* Requires the vectorise the evaluation, which is not always easy.

## Calling sequence

```python
def mySimulator (x):
    [...]
    return y
myWrapper=PythonFunction(nbInputs, nbOutputs, func_sample=mySimulator)
```

where:

* `x`: the input of the simulator, a `Sample` with size `nbExperiments` (`getSize()`), and dimension `nbInputs` (`getDimension()`),
* `y`: the output of the function. Can be:
    * a `numpy.array` with `nbExperiments` rows and `nbOutputs` columns
    * a `ot.Sample` with size equal to `nbExperiments` and dimension equal to `nbOutputs`

## Vectorised PythonFunction: example with Numpy


In [5]:
def mySimulatorVect(x):
    # Convert Sample > Numpy array
    x = np.array(x)
    x0 = x[:, 0]  # Get column 0
    x1 = x[:, 1]
    x2 = x[:, 2]
    y0 = x0 + x1 + x2
    y1 = x0 - x1 * x2
    y = np.vstack((y0, y1))  # Stack two rows
    y = y.transpose()  # Transpose the result
    return y


myWrapperVect = ot.PythonFunction(3, 2, func_sample=mySimulatorVect)

In [6]:
outputVect = ot.CompositeRandomVector(myWrapperVect, inputRandomVector)
montecarlosize = 10000
outputSample = outputVect.getSample(montecarlosize)

empiricalMean = outputSample.computeMean()
print(empiricalMean)
empiricalSd = outputSample.computeStandardDeviation()
print(empiricalSd)

[-0.00077008,-0.0165189]
[1.74231,1.40859]


## MemoizeFunction to manage the history of evaluations

The `MemoizeFunction` class defines a system to manage the history of the evaluations of the function.

| Method | Feature |
|-|-|
| `enableHistory()` | enables the history (default : enabled) | |
| `disableHistory()` | disable the history |
| `isHistoryEnabled()` | `True` is the history is enabled |
| `clearHistory()` | delete the history |
| `getHistoryInput()` | a `Sample`, the history of the input `X` |
| `getHistoryOutput()` | a `Sample`, the history of the outputs `Y` |

**Table 3.** The main methods of the `MemoizeFunction` class.

In the next script, we create a function and wrap it into a `MemoizeFunction`. Then we generate a sample on output of the function. 

In [7]:
myWrapper = ot.PythonFunction(3, 2, mySimulator)
myWrapper = ot.MemoizeFunction(myWrapper)

outputVariableOfInterest = ot.CompositeRandomVector(myWrapper, inputRandomVector)
montecarlosize = 10
outputSample = outputVariableOfInterest.getSample(montecarlosize)

The next cell gets the input history: this corresponds to the 10 outputs of the previous Monte-Carlo sample.

In [8]:
inputs = myWrapper.getInputHistory()
inputs

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=10 dimension=3 data=[[-0.342605,-0.689637,1.24309],[-2.13865,-1.46352,1.10263],[-0.621272,0.0568358,-1.01041],[0.140946,0.553279,0.690434],[0.305662,-1.1365,1.69223],[-1.5188,0.969533,-1.76564],[1.86033,1.21488,0.812872],[0.327706,-0.688817,-2.01179],[-0.395283,1.84128,-1.36851],[0.456102,-0.131374,0.57282]]

Get the output history.

In [9]:
outputs = myWrapper.getOutputHistory()
outputs

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=10 dimension=2 data=[[0.210845,0.514674],[-2.49955,-0.524936],[-1.57485,-0.563844],[1.38466,-0.241056],[0.861384,2.22889],[-2.31491,0.193046],[3.88808,0.872784],[-2.3729,-1.05805],[0.0774926,2.12452],[0.897547,0.531356]]

##  Exercises


### Exercise 1: a function with 4 inputs

We consider a new model, with a new input $X_4$ and a new output $Y_3$:

$$
\begin{eqnarray}
Y_1 &=& X_1 + X_2 + X_3 \\
Y_2 &=& X_1 − X_2 X_3 \\
Y_3 &=& 2 X_1 + 3 X_2 + 4 X_4
\end{eqnarray}
$$

**Questions**

* Update the Python function to simulate the new model.
* Add a new random variable `X4` with standard normal distribution into the probabilistic model.
* Estimate the sample mean using simple Monte-Carlo sampling.


### Solution of exercise 1: a function with 4 inputs


In [10]:
def mySimulator(x):
    y0 = x[0] + x[1] + x[2]
    y1 = x[0] - x[1] * x[2]
    y2 = 2 * x[0] + 3 * x[1] + 4 * x[3]
    y = [y0, y1, y2]
    return y


myWrapper = ot.PythonFunction(4, 3, mySimulator)
# Create the marginal distributions
X0 = ot.Normal(0.0, 1.0)
X1 = ot.Normal(0.0, 1.0)
X2 = ot.Normal(0.0, 1.0)
X3 = ot.Normal(0.0, 1.0)
# Create the input probability distribution
inputDistribution = ot.ComposedDistribution((X0, X1, X2, X3))
# Create the input random vector
inputRandomVector = ot.RandomVector(inputDistribution)
# Create the output variable of interest
outputVariableOfInterest = ot.CompositeRandomVector(myWrapper, inputRandomVector)
# Probabilistic Study: central dispersion
montecarlosize = 10000
# Start the simulations
outputSample = outputVariableOfInterest.getSample(montecarlosize)
# Get the empirical mean and standard deviations
outputDim = myWrapper.getOutputDimension()
outputSample.computeMean()

class=Point name=Unnamed dimension=3 values=[-0.000337898,-0.00609942,0.115035]

### Exercise 2: gradient of a Python function

OpenTURNS can evaluate the derivative of a Python function using finite differences. We can customize the finite difference fomula and the step. Moreover, when the Jacobian matrix is available in a Python function, we can give that function to the library so that it can be used. This can be convenient when for example the exact derivative is known. 

**Questions**

* Define a function `myWrapper` as in the previous example.
* Use the `gradient()` method of `myWrapper` to evaluate the gradient $g'(x)$ at point $x = (1, 2, 3)^T$. 
* Use the `hessian()` of `myWrapper` to evaluate the Hessian matrix.
* Use the next script to configure the gradient using a non-centered finite difference formula with a step size equal to $h = 10^{-2}$. 

```python
wrapImpl = myWrapper.getEvaluation()
h = 1.e-2
myGradient = ot.NonCenteredFiniteDifferenceGradient(h, wrapImpl)
myWrapper.setGradient(myGradient)
```

* Evaluate the gradient with the `gradient()` method and compare with the previous result.
* We can give to the library a Python function which evaluates the gradient. To do this, we use:
```python
myWrapper = ot.PythonFunction(nbInputs, nbOutputs, mySimulator, gradient=mySimulatorGradient)
```
where `mySimulatorGradient` is a function which evaluates the gradient.
Compute by hand (or using [Wolfram Alpha](https://www.wolframalpha.com/input?i=derivative+of+x1+%2B+x2+%2B+x3+with+respect+to+x1)). Define the function `mySimulatorGradient` which evaluates the Jacobian matrix. Since there are 3 input variables, the `list` returned by `mySimulatorGradient` must contain 3 elements. Each element of the `list` must be a `list` with 2 items representing the derivatives of each output. Finally, create the function using the `gradient` option.


### Solution de l'exercise 2: gradient of a Python function

In [11]:
def mySimulator(x):
    y0 = x[0] + x[1] + x[2]
    y1 = x[0] - x[1] * x[2]
    y = [y0, y1]
    return y


inputDim = 3
outputDim = 2
myWrapper = ot.PythonFunction(inputDim, outputDim, mySimulator)

# Evaluate the gradient
d = myWrapper.gradient([1, 2, 3])
print("type(d)=", type(d))  # OT Matrix
print("Gradient par DF=")
print(d)

# Evaluate the Hessian
dd = myWrapper.hessian([1, 2, 3])
print("type(dd)=", type(dd))  # OT SymmetricTensor
print("Hessienne=")
print(dd)

# Configure the finite difference formula of the gradient
wrapImpl = myWrapper.getEvaluation()
myGradient = ot.NonCenteredFiniteDifferenceGradient(1.0e-2, wrapImpl)
myWrapper.setGradient(myGradient)

d = myWrapper.gradient([1, 2, 3])
print("Gradient par DF non centrée=")
print(d)

type(d)= <class 'openturns.typ.Matrix'>
Gradient par DF=
[[  1  1 ]
 [  1 -3 ]
 [  1 -2 ]]
type(dd)= <class 'openturns.typ.SymmetricTensor'>
Hessienne=
sheet #0
[[  0            2.22045e-08  0           ]
 [  2.22045e-08  0            2.22045e-08 ]
 [  0            2.22045e-08  0           ]]
sheet #1
[[  0            0            0           ]
 [  0            0           -1           ]
 [  0           -1            0           ]]
Gradient par DF non centrée=
[[  1  1 ]
 [  1 -3 ]
 [  1 -2 ]]


In [12]:
# Configure the gradient of a Python function
def mySimulatorGradient(x):
    dyx0 = [1.0, 1.0]
    dyx1 = [1.0, -x[2]]
    dyx2 = [1.0, -x[1]]
    y = [dyx0, dyx1, dyx2]
    return y


myWrapper = ot.PythonFunction(3, 2, mySimulator, gradient=mySimulatorGradient)
d = myWrapper.gradient([1, 2, 3])
print("d - Exact =")
print(d)

d - Exact =
[[  1  1 ]
 [  1 -3 ]
 [  1 -2 ]]


### Exercise 3: manage the history of a Python function

**Questions**

* See the change in the output of the `isHistoryEnabled()` method.
* What are the methods which allows to get the history of the input and output values? Experiment with them.
* How to get the number of evaluations of the function? How is this changed when several algorithms are used in sequence?
* Use the `clearHistory()` method and check that the history is empty after the call.

### Solution of l'exercise 3: manage the history of a Python function

In [13]:
def mySimulator(x):
    y0 = x[0] + x[1] + x[2]
    y1 = x[0] - x[1] * x[2]
    y = [y0, y1]
    return y


myWrapper = ot.PythonFunction(3, 2, mySimulator)
myWrapper = ot.MemoizeFunction(myWrapper)

# Create the marginal distributions
X0 = ot.Normal(0.0, 1.0)
X1 = ot.Normal(0.0, 1.0)
X2 = ot.Normal(0.0, 1.0)

# Create the input probability distribution
inputDistribution = ot.ComposedDistribution((X0, X1, X2))
# Create the input random vector
inputRandomVector = ot.RandomVector(inputDistribution)
# Create the output variable of interest
outputVariableOfInterest = ot.CompositeRandomVector(myWrapper, inputRandomVector)
# Probabilistic Study: central dispersion
montecarlosize = 20
outputSample = outputVariableOfInterest.getSample(montecarlosize)

# Get the history
inputs = myWrapper.getInputHistory()
print("inputs")
print(inputs[:5])
outputs = myWrapper.getOutputHistory()
print("outputs")
print(outputs[:5])
# Nombre d'appels à la fonction G
nGEvals = inputs.getSize()
print("nGEvals = %d" % (nGEvals))

# Clear the history
myWrapper.clearHistory()

# See how the history is now empty
print("After clearHistory. Output history:")
myWrapper.getOutputHistory()

inputs
    [ v0         v1         v2         ]
0 : [ -0.222044  -0.770033   1.16119   ]
1 : [  0.978763  -0.154068  -0.788095  ]
2 : [ -0.907291   0.0717698  1.6041    ]
3 : [ -0.621793   0.328243   0.540337  ]
4 : [  0.504746  -0.434332  -0.852683  ]
outputs
    [ v0         v1         ]
0 : [  0.169115   0.672112  ]
1 : [  0.0366002  0.857343  ]
2 : [  0.768574  -1.02242   ]
3 : [  0.246787  -0.799155  ]
4 : [ -0.782268   0.134399  ]
nGEvals = 20
After clearHistory. Output history:


class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=0 dimension=2 data=[]

### Exercise 4: benchmark

See `wrapper-python-benchmark.py` in the same directory.

### Exercise 5: configure the number of CPUs

See the script `test_n_cpus.py`.

### Exercise 6: use otwrapy

**Questions**

- Install `otwrapy`.
- Read the doc [of the Parallelizer class](https://openturns.github.io/otwrapy/master/_generated/otwrapy.Parallelizer.html#otwrapy.Parallelizer). 
- Parallelize the function with `n_cpus`.
- Experiment with the 4 options of the optional `backend` input argument : "ipyparallel", "joblib", "pathos", or "multiprocessing".

### Exercise 7: use Jax

**Questions**

- Install Jax
- Read the [Quickstart](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html). 
- Compute the exact derivative of the `Ishigami` function.

In [14]:
def ishigami(x):
    x1, x2, x3 = x
    a = 7.0
    b = 0.1
    y = np.sin(x1) + a * np.sin(x2) ** 2 + b * x3**4 * np.sin(x1)
    return [y]


ishigami([1.0] * 3)

[5.882132011203685]